In [34]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import os
from pathlib import Path

%matplotlib inline

sns.set_style("whitegrid")
plt.rcParams["figure.figsize"] = (10, 6)


In [35]:
# Verificación de archivos disponibles
print("Archivos disponibles en el directorio actual:")
archivos_disponibles = [f for f in os.listdir() if f.endswith('.csv')]
for archivo in archivos_disponibles:
    print(f" - {archivo}")

Archivos disponibles en el directorio actual:
 - NBA_PBP_2017-18.csv
 - NBA_PBP_2019-20.csv
 - play_by_play.csv
 - player.csv
 - other_stats.csv
 - clutch_stats.csv
 - game_summary.csv
 - NBA_PBP_2016-17.csv
 - NBA_PBP_2018-19.csv
 - NBA_PBP_2020-21.csv
 - draft_history.csv
 - common_player_info.csv
 - NBA_PBP_2015-16.csv
 - game.csv
